In [1]:
# Run this cell to automatically clone the repo
import sys
import os

if 'google.colab' in sys.modules:
    if not os.path.exists("/content/a1003"):
        !git clone "https://github.com/pkyoung/a1003.git" /content/a1003
    %cd /content/a1003/local
else:
    if not os.path.exists("~/a1003"):
        !git clone "https://github.com/pkyoung/a1003.git" ~/a1003
    %cd ~/a1003/local

if not os.path.exists("data/a1003"):
  !git clone https://hf.co/datasets/pkyoung/a1003.git ./data/a1003

/content/a1003/local


huggingface에서 테스트용 데이터셋을 내려받습니다.
9명의 화자가 20개의 짧은 문장을 발성한 wav 파일과 정답전사문 파일입니다.


In [2]:
if not os.path.exists("data/a1003"):
  !git clone https://hf.co/datasets/pkyoung/a1003.git ./data/a1003

내려받은 파일을 들어봅니다.
내려받은 디렉토리의 구조는 다음과 같습니다.
```
data/a1003
    ├── README.md
    ├── result_whisper_turbo.txt
    ├── text
    ├── wav.scp
    ├── wav
    │   ├── spk1
    │   │   ├── 1.wav
    │   │   ├── 10.wav
    │   │   ├── 11.wav
    ...
```
`text` 파일과 `wav.scp` 파일은 앞서 설명한 `kaldi` 파일과 같은 형식을 갖습니다. 파일은 다음과 같은 방법으로 들어볼 수 있습니다.

In [3]:
from IPython.display import Audio
filename = "data/a1003/wav/spk1/1.wav"
Audio(filename, autoplay=False)

In [4]:
pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 39.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s 

## Whipser 음성인식

한개의 wav 파일에 대해서 인식을 수행해봅니다. 간단하게 파일을 입력으로 `whisper` 명령어를 실행합니다.

In [5]:
!whisper data/a1003/wav/spk1/1.wav

100%|██████████████████████████████████████| 1.51G/1.51G [00:06<00:00, 236MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Korean
[00:00.000 --> 00:08.080]  위키피디아에 따르면 소리 또는 음 또는 음파는 공기나 물 같은 매질의 진동을 통해 전달되는 종바라고 합니다.


whisper는 task로 transcribe와 translate 두가지를 할 수 있습니다. 인식에 사용할 모델, 인식대상 언어를 지정하여 음성인식을 수행합니다. (Colab 환경에서 약 5분 이상 소요)

In [15]:
!whisper --model large-v3 --language ko --task transcribe data/a1003/wav/spk6/recording_17.wav

100%|██████████████████████████████████████| 2.88G/2.88G [00:23<00:00, 132MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
[00:00.000 --> 00:07.040]  자유로운 날개짓 활활활 꽃송이가 나를 삼킬 걸 알면서 난 뛰어들었어
[00:07.040 --> 00:37.020]  자막 제공 및 영상 제공 및 광고를 포함하고 있습니다.


파이선 코드 안에서는 다음과 같은 방법으로 이용할 수 있습니다. Low-level 핸들링을 위해서는 `transcribe()` 함수 외에 `decode()` 함수를 사용할 수 있습니다.

In [7]:
import whisper
model = whisper.load_model("tiny")
result = model.transcribe("data/a1003/wav/spk6/recording_17.wav")
print(result["text"])

100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 207MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 자유로운 날개지 더울허허할 꽃순이가 날을 삼켓거라 이멘서 난 뛰어들었어.


전체 테스트 데이터셋 (180개) 에 대해서 인식을 수행해서 결과를 `result.txt` 파일에 저장합니다.

보통의 GPU 카드가 설치된 머신에서는 수 분 이내로 인식이 완료됩니다. Colab 환경에서는 10분 이상 소요될 수 있습니다.

In [8]:
import whisper
import tqdm
import warnings
warnings.filterwarnings("ignore")

model = whisper.load_model("tiny")

with open('data/a1003/wav.scp') as fscp:
    lines = fscp.readlines()

with open("result.txt", "w") as f:
  for line in  tqdm.tqdm(lines, desc="Transcribing", unit="utterance"):
    uttid, filepath = line.split()
    result = model.transcribe(filepath)
    print(uttid, result["text"], file=f)

Transcribing:  50%|█████     | 90/180 [04:05<04:05,  2.73s/utterance]


KeyboardInterrupt: 

저장된 인식결과를 확인합니다.

In [9]:
!head result.txt

spk1/1  위키디따르멘 소리 또는 음 또는 음파는 공기나 물 같은 매질의 진동을 통해 전달되는 종발하고 합니다.
spk1/10  파도는 반복적으로 진동하는 신호를 나타내기 때문에 우는 소리를 웨이폼 형태로 나타날 수 있습니다.
spk1/11  프리에 변환을 통해 어둠 스페트럼은 시간 정보가 없어지게 됩니다.
spk1/12  파형을 일정한 특정 길에 프레임으로 잘라서 각 프레임마다 프리에 변환을 취하고 스펙러움을 구하는 방법을 취합니다.
spk1/13  이렇게 음성 전체로부터 어둠 여러 개의 스페트럼을 시간 주게 나열하면 시간 변화에 다른 스페트럼의 변화를 스페트로그램으로 정의합니다.
spk1/14  스펙트로그램은 소리나, 바동을 시각하여 파악하기 위한 도구로, 파영과 스펙트로 매트쯤이 좋았대요.
spk1/15  사람의 청각기간은 고주파보다 저주파대역에서 더 민감하다고 합니다.
spk1/16  2월 2일 달팽이 간의 구조로 살펴보면 달팽이 간의 가장 안쪽 청각색포는 저주파 대역을 잔지합니다.
spk1/17  바깥쪽 청각 세포는 고주파 대역을 인지한다는 사실을 통해 모든 주파수 대역을 같은 비중으로 인지하지 않습니다.
spk1/18  고주파에서 서주파로 내려갈수록 담당하는 주파수를 대역이 점점 더 도미레인다는 저의입니다.


## Whisper 모델의 음성인식 오류율 측정
발화별 CER과 평균 CER을 계산합니다.

`data/a1003/result_whisper_turbo.txt` 파일에 `large-v3-turbo` 모델의
인식 결과를 미리 만들어 두었습니다. 이 파일을 사용해서 인식률을 측정합니다.

In [10]:
!python uttcer.py data/a1003/text data/a1003/result_whisper_turbo.txt

spk1/1  4.55 2 44
spk1/10  4.65 2 43
spk1/11 14.81 4 27
spk1/12  8.16 4 49
spk1/13  1.75 1 57
spk1/14  2.17 1 46
spk1/15  0.00 0 29
spk1/16  2.33 1 43
spk1/17  2.17 1 46
spk1/18  2.70 1 37
spk1/19  3.45 1 29
spk1/2  2.78 1 36
spk1/20  2.33 1 43
spk1/3  3.33 1 30
spk1/4  5.00 1 20
spk1/5  2.17 1 46
spk1/6 11.11 4 36
spk1/7  1.85 1 54
spk1/8  1.82 1 55
spk1/9  5.13 2 39
spk2/utt1 25.00 4 16
spk2/utt10  7.69 1 13
spk2/utt11  7.14 1 14
spk2/utt12 235.29 40 17
spk2/utt13  8.57 3 35
spk2/utt14 10.00 1 10
spk2/utt15  5.56 1 18
spk2/utt16 13.33 2 15
spk2/utt17 16.67 4 24
spk2/utt18 22.22 4 18
spk2/utt19 17.86 5 28
spk2/utt2 17.65 3 17
spk2/utt20 10.00 1 10
spk2/utt3  8.33 1 12
spk2/utt4 33.33 7 21
spk2/utt5  9.68 3 31
spk2/utt6 33.33 5 15
spk2/utt7 30.00 6 20
spk2/utt8 21.74 5 23
spk2/utt9 17.95 7 39
spk3/data1  2.86 1 35
spk3/data10 50.00 20 40
spk3/data11  9.52 4 42
spk3/data12  2.78 1 36
spk3/data13  3.23 1 31
spk3/data14  6.45 2 31
spk3/data15 44.83 13 29
spk3/data16 14.29 1 7
spk3/data17 

화자별 음성인식 성능을 계산합니다.

In [11]:
%%bash
for s in spk{1..9}; do
  echo -n "$s " >&2
  python uttcer.py data/a1003/text <(grep "$s/" data/a1003/result_whisper_turbo.txt) \
    > /dev/null
done


spk1 N= 809 E= 31 CER=  3.83
spk2 N= 396 E= 104 CER= 26.26
spk3 N= 610 E= 71 CER= 11.64
spk4 N= 1360 E= 109 CER=  8.01
spk5 N= 981 E= 37 CER=  3.77
spk6 N= 515 E= 25 CER=  4.85
spk7 N= 515 E= 92 CER= 17.86
spk8 N= 848 E= 59 CER=  6.96
spk9 N= 995 E= 55 CER=  5.53


평균 오류율이 높은 화자에 대해서는 몇몇 파일을 들어봅니다. 주석처리된 부분을 바꿔가면서 들어봅니다.

In [12]:
from IPython.display import Audio
filename = "data/a1003/wav/spk2/utt1.wav"
#filename = "data/a1003/wav/spk7/recording_00.wav"
#filename = "data/a1003/wav/spk3/data15.wav"

Audio(filename, autoplay=False)

문장부호를 제외한 CER을 계산합니다.

In [14]:
!python uttwer.py <(python pp.py -s data/a1003/text) \
  <(python pp.py -s data/a1003/result_whisper_turbo.txt) > /dev/null

N= 6835 E= 346 WER=  5.06


## Whisper 모델별 음성인식 성능 및 속도

각 모델별 모델크기, 인식 소요 시간 및 인식 성능은 다음 표와 같습니다.
인식소요시간 및 인식성능은 환경에 따라 달라질 수 있습니다. (WER1, CER1: 문장부호 제외 전/WER2, CER2: 문장부호 제외 후)

| Model          | Size | Elapsed | RTF | WER1 | CER1 | WER2 | CER2 |
| ---            | ---  | ---     | --- | ---  | ---  | ---  | ---  |
| tiny           | 73M  | 1m6s    |0.04 |50.11 |26.62 |44.10 |24.55 |
| base           | 139M | 1m2s    |0.04 |38.94 |17.74 |32.76 |15.61 |
| small          | 462M | 1m38s   |0.06 |32.73 |13.07 |24.55 |10.13 |
| medium         | 1.5G | 3m5s    |0.11 |29.57 |11.14 |21.90 | 8.46 |
| large-v1       | 2.9G | 4m0s    |0.15 |25.33 | 9.03 |16.98 | 6.15 |
| large-v2       | 2.9G | 3m46s   |0.14 |30.34 |13.43 |21.64 |10.24 |
| large-v3       | 2.9G | 4m1s    |0.15 |29.44 |13.46 |20.40 |10.43 |
| large-v3-turbo | 1.6G | 1m19s   |0.05 |24.18 | 8.29 |14.67 | 5.05 |
